In [ ]:
from tripadvisor import Tripadvisor

hotel_id = "23770286"

tripadvisor = Tripadvisor()
# hotels = tripadvisor.get_hotels()
hotel = tripadvisor.hotel_detail(hotel_id)

In [ ]:
import json

print(json.dumps(hotel, indent=4))

In [ ]:
from tripadvisor import Tripadvisor

hotel_id = "20226478"

tripadvisor = Tripadvisor()
review = tripadvisor.hotel_review(hotel_id, 0)

In [ ]:
import json

print(json.dumps(review['data'][6], indent=7))
# print(len(review['data']))

In [ ]:
import json

beneran_review = [data for data in review['data'] if 'ad_context' not in data]
len(beneran_review)

In [ ]:
review2 = tripadvisor.hotel_review(hotel_id, 20)

In [ ]:
print(json.dumps(review2['data'][0], indent=7))

In [33]:
from tripadvisor import Tripadvisor
import mysql.connector
import time
import config
import os
import json

hotel_data_dir = './hotel_data'

# initiate mysql
db = init_db()
cursor = get_cursor(db)

# select all hotel list that have not been parsed
hotel_list = db_select(db, "SELECT * FROM hotel_list WHERE hotel_detail_parsed=0 LIMIT 254")

# instantiate TripAdvisor object
tripadvisor = Tripadvisor()

# iterate per hotel to get hotel detail then change hotel_detail_parsed to True
for hotel in hotel_list:
    hotel_id, hotel_link, hotel_detail_parsed, hotel_review_parsed = hotel

    # create hotel dir if not exist
    hotel_id_dir = os.path.join(hotel_data_dir, hotel_id)
    if not os.path.exists(hotel_id_dir):
        os.makedirs(hotel_id_dir)

    # get hotel detail from tripadvisor api
    hotel_detail = tripadvisor.hotel_detail(hotel_id)

    # store hotel_detail data to database and local file
    if 'errors' in hotel_detail.keys():
            print(f'can\'t parse hotel with id {hotel["hotel_id"]}')
    else:
        #store and update database
        cursor.execute(
            'INSERT IGNORE INTO hotel_detail (hotel_id, hotel_detail) VALUES (%s, %s)',
            (hotel_id, json.dumps(hotel_detail))
        )
        cursor.execute(
            f'UPDATE hotel_list SET hotel_detail_parsed=1 WHERE hotel_id="{hotel_id}"'
        )
        db.commit()

        #store to local file
        hotel_detail_path = os.path.join(hotel_id_dir, f'{hotel_id}_hotel_detail.json') # hotel_detail local file path
        with open(hotel_detail_path, 'w') as json_file:
            json.dump(hotel_detail, json_file, indent=7)
    time.sleep(5)
    
    
def init_db():
    return mysql.connector.connect(
            host = config.configSQLCrawler.HOST,
            user = config.configSQLCrawler.USER,
            passwd = config.configSQLCrawler.PASSWD,
            database = config.configSQLCrawler.DATABASE,
            port = config.configSQLCrawler.PORT,
            auth_plugin=config.configSQLCrawler.AUTH_PLUGIN
        )

def get_cursor(con):
    try:
        con.ping(reconnect=True, attempts=3, delay=5)
    except mysql.connector.Error as e:
        raise Exception('db error')
    return con.cursor(buffered=True)

def db_select(con, query):
    try:
        cursor = get_cursor(con)
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Exception as e:
        print(e)
    finally:
        cursor.close()



254


# ALTER JSON FILE AND UPLOAD TO MYSQL SERVER (ARCHIVED)

In [27]:
import mysql.connector
import config
import json



destination_path = './hotel_data/hotel_list.json'

db = init_db()
cursor = get_cursor(db)

temp_list = list()
with open('hotel_list.json') as json_file:
    json_data = json.load(json_file)
    
    for hotel in json_data:
        cursor.execute(
            "INSERT IGNORE INTO hotel_list (hotel_id, hotel_link, hotel_detail_parsed, hotel_review_parsed) VALUES (%s, %s, %s, %s);",
            (hotel['hotel_id'], hotel['link'], False, False )
            )
        db.commit()

def init_db():
    return mysql.connector.connect(
            host = config.configSQLCrawler.HOST,
            user = config.configSQLCrawler.USER,
            passwd = config.configSQLCrawler.PASSWD,
            database = config.configSQLCrawler.DATABASE,
            port = config.configSQLCrawler.PORT,
            auth_plugin=config.configSQLCrawler.AUTH_PLUGIN
        )

def get_cursor(con):
    try:
        con.ping(reconnect=True, attempts=3, delay=5)
    except mysql.connector.Error as e:
        raise Exception('db error')
    return con.cursor(buffered=True)

def db_select(con, query):
    try:
        cursor = get_cursor(con)
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Exception as e:
        print(e)
    finally:
        cursor.close()

